In [ ]:
!pip install numpy tqdm scikit-learn tensorflow keras

In [5]:
# Import packages needed
import os
import numpy as np
import keras

from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [ ]:
# Function for reading all the image from the dataset folder
def get_data():
    # read all the folders
    data_path = "../dataset/Musinsa_dataset"
    folders = os.listdir(data_path)

    # read all the images inside the folders
    style2index = []
    images = []
    labels = []
    for i in range(len(folders)):
        folder = folders[i]
        folder_path = f"{data_path}/{folder}"

        if not os.path.isdir(folder_path):
            continue

        files = os.listdir(folder_path)
        print(folder)

        for file in tqdm(files):
            try:
                image = keras.preprocessing.image.load_img(data_path + "/" + folder + "/" + file, target_size=(224, 224))
                image = keras.preprocessing.image.img_to_array(image)
                images.append(image)
                labels.append(i)
            except Exception as e:
                print(f"Error reading image {file}: {e}")
                continue
        style2index.append({folder: i})

    return images, labels, style2index

In [8]:
image, labels, style2index = get_data()
print(len(image), len(labels))
print(style2index)

FileNotFoundError: [Errno 2] No such file or directory: '../data'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=0.2, random_state=42)

print(f"train set size: {len(X_train)}, {len(y_train)}")
print(f"test set size: {len(X_test)}, {len(y_test)}")

In [ ]:
X_train = np.array([np.array(image).reshape(224, 224, 3) for image in X_train])
X_test = np.array([np.array(image).reshape(224, 224, 3) for image in X_test])

In [ ]:
def generate_pair(X, y):
    """
        For contrastive learning, we need the dataset in pair.
        There should exist 
        Input: X(image), y(label)
        Output: X_pairs(image pair), y_pairs(label pair)
    """
    X_pairs = []
    y_pairs = []

    for i in range(len(X)):
        digit = y[i]
        
        positive_digit_index = np.random.choice(np.where(y==digit)[0])
        X_pairs.append([X[i], X[positive_digit_index]])
        y_pairs.append([0])

        negative_digit_index = np.random.choice(np.where(y!=digit)[0])
        X_pairs.append(X[i], X[negative_digit_index])
        y_pairs.append([1])

    indices = np.arange(len(X_pairs))
    np.random.shuffle(indices)

    return np.array(X_pairs)[indices], np.array(y_pairs)[indices]

In [ ]:
X_train_pairs, Y_train_pairs = generate_pair(X_train, y_train)
X_test_pairs, Y_test_pairs = generate_pair(X_test, y_test)

print("X_train_pairs shape: ", X_train_pairs.shape)
print("X_test_pairs shape: ", X_test_pairs.shape)